In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('weatherAUS.csv')
df.head()

In [4]:
date = pd.to_datetime(df['Date'])
df['Day'] = date.dt.day
df['Month'] = date.dt.month
df['Year'] = date.dt.year

In [6]:
dict_lat = [36.07, 33.90, 31.48, 30.32, 29.49, 32.92, 33.28, 29.04, 33.72, 33.60, 33.86, 33.95, 35.16, 32.79, 34.37, 35.31, 35.42, 35.53, 37.51,36.74, 38.12, 37.67, 37.83, 34.24, 36.31, 38.31, 37.74, 
            37.92, 27.48, 16.87,27.94, 19.25, 34.95, 37.75, 34.48, 31.16, 35.03, 34.03, 31.67, 31.93, 31.92,32.99, 34.95, 42.89, 41.42, 23.80, 12.42, 14.52, 25.19] 

dict_long = [146.95, 150.73, 145.83, 153.12, 149.85, 151.80, 151.58,167.94, 150.68, 150.78, 151.20, 151.17, 147.46, 151.84, 150.93, 149.20, 149.09,148.77, 143.79, 144.33, 147.13, 144.83, 
             144.98, 142.09, 141.65, 141.47, 145.10,141.26, 153.04, 145.75, 153.43, 146.77, 138.52, 140.77, 139.01, 136.81, 117.88,115.06, 116.02, 115.98, 115.87, 121.62, 116.72, 147.33, 147.12, 
             133.89, 130.89,132.38, 130.97]

dict_elev = [164, 81, 260, 4, 213, 33, 19, 112, 25, 19, 43, 6, 212, 8, 10, 577, 587, 1760, 435, 209, 5, 113, 8, 50, 139, 81, 66, 51, 8, 2, 3, 4, 2, 63, 275, 167, 3, 85, 40, 15, 25, 249, 73, 51, 5, 546, 30, 134, 492]


df['Location_Latitude'] = pd.DataFrame(data = dict_lat)
df['Location_Longitude'] = pd.DataFrame(data = dict_long)
df['Location_Elevation'] = pd.DataFrame(data = dict_elev)

In [7]:
df = df.fillna(df.mean())
for i in df[["WindGustDir","WindDir9am","WindDir3pm"]]:
    df[i] = df[i].fillna(df[i].mode()[0])

C:\Users\Krbli\AppData\Local\Temp\ipykernel_14852\3244452998.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


In [8]:
for i in df[["WindGustDir","WindDir9am","WindDir3pm"]]:
        df[i]=df[i].replace(['E','ENE','NE','NNE','N','NNW','NW','WNW','W','WSW','SW','SSW','S','SSE','SE','ESE'],
                            [j*np.pi/8 for j in range(16)])
        
for i in df[["WindGustDir","WindDir9am","WindDir3pm"]]:
        df[str(i)+"_cos"] = np.round(np.cos(df[i]),2)
        df[str(i)+"_sin"] = np.round(np.sin(df[i]),2)
        
df['RainToday']=df.RainToday.replace(["Yes","No"],[1,0])
df['RainTomorrow']=df.RainTomorrow.replace(["Yes","No"],[1,0])

In [9]:
df = df.drop(['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)


In [10]:
df = df.dropna(axis=0, how='any', subset = ['RainToday','RainTomorrow'])

In [11]:
df.isna().sum()

MinTemp               0
MaxTemp               0
Rainfall              0
Evaporation           0
Sunshine              0
WindGustSpeed         0
WindSpeed9am          0
WindSpeed3pm          0
Humidity9am           0
Humidity3pm           0
Pressure9am           0
Pressure3pm           0
Cloud9am              0
Cloud3pm              0
Temp9am               0
Temp3pm               0
RainToday             0
RainTomorrow          0
Day                   0
Month                 0
Year                  0
Location_Latitude     0
Location_Longitude    0
Location_Elevation    0
WindGustDir_cos       0
WindGustDir_sin       0
WindDir9am_cos        0
WindDir9am_sin        0
WindDir3pm_cos        0
WindDir3pm_sin        0
dtype: int64

In [12]:
target = df['RainTomorrow']
data = df.drop('RainTomorrow', axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state=120)

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [14]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

In [15]:
pd.crosstab(y_test, y_pred, rownames = ['Reality'], colnames = ['Prediction'])

Prediction,0.0,1.0
Reality,,
0.0,20829,1147
1.0,3088,3094


In [16]:
print('score_train:', lr.score(X_train, y_train))
print('score_test:', lr.score(X_test, y_test))

score_train: 0.8442940983228121
score_test: 0.8495986930889978


In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score

print('precision:',precision_score(y_test, y_pred))
print('recall:', recall_score(y_test, y_pred))
print('f1:', f1_score(y_test, y_pred))


precision: 0.7295449186512615
recall: 0.5004852798447105
f1: 0.5936870382807254


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.87      0.95      0.91     21976
         1.0       0.73      0.50      0.59      6182

    accuracy                           0.85     28158
   macro avg       0.80      0.72      0.75     28158
weighted avg       0.84      0.85      0.84     28158

